#TP deep learning sous attaque adversaire (2024)
####Adrien Chan-Hon-Tong
####TP réalisé à partir de résultats de Pol Labarbarie


L'objet de ce TP est de démontrer
- la faciliter de produire des attaques adversaires "white box" sur des réseaux naifs quelles soient invisibles ou par patch
- mais que cela est beaucoup plus dur sur un réseau robustifier (cas invisible)
- ou encore qu'il est beaucoup plus difficile de produire des attaques "transferable"

## generalité
Commençons par télécharger 10 images d'imagenet.

In [5]:
!rm -f *
!wget https://httpmail.onera.fr/21/9f6c7025f0680226eb94c7a73cc4290dG7fRIu/data.zip
!unzip data.zip
!ls

rm: cannot remove 'sample_data': Is a directory
--2025-01-27 22:41:14--  https://httpmail.onera.fr/21/9f6c7025f0680226eb94c7a73cc4290dG7fRIu/data.zip
Resolving httpmail.onera.fr (httpmail.onera.fr)... 144.204.16.9
Connecting to httpmail.onera.fr (httpmail.onera.fr)|144.204.16.9|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-01-27 22:41:15 ERROR 404: Not Found.

unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.
sample_data


Affichons les : les 5 premières sont des "avions" et les 5 suivantes des "requins"

In [4]:
import torch
import torchvision
import matplotlib.pyplot as plt

x = [torchvision.io.read_image(str(i)+".png") for i in range(10)]
x = torch.stack(x,dim=0).float()/255

visu = torchvision.utils.make_grid(x, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

RuntimeError: [Errno 2] No such file or directory: '1.png'

In [ ]:
SHARK, PLANE = [2, 3, 4], [403, 404, 405]
normalize = torchvision.transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
resnet = torchvision.models.resnet101(
    weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
).eval()

with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 117MB/s]


tensor([403, 405, 404, 405, 404,   4,   3,   3,   3,   2])


On voit que le réseau classe correctement ces images.

## Attaque standard "white box"

On va maintenant rajouter à ces images un petit bruit "invisible" pour l'oeil mais perturbant pour le réseau.

In [ ]:
y = torch.Tensor([403, 405, 404, 405, 404,   4,   3,   3,   3,   2]).long()
cefunction = torch.nn.CrossEntropyLoss()
attaque = torch.nn.Parameter(torch.zeros(x.shape))
optimizer = torch.optim.SGD([attaque],lr=0.005)
for i in range(10):
  z = resnet(normalize(x+attaque))
  ce = cefunction(z,y)
  print(i,float(ce))
  ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
  optimizer.zero_grad()
  ce.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
      # l'attaque doit être invisible
      attaque = torch.clamp(attaque, -10./255,+10./255)

      # attaque+x doit être entre 0 et 1
      lowbound = -x
      uppbound = 1-x
      attaque = lowbound*(attaque<lowbound).float() + uppbound*(attaque>uppbound).float() + attaque *(attaque>=lowbound).float()*(attaque<=uppbound).float()

  attaque = torch.nn.Parameter(attaque.clone())
  optimizer = torch.optim.SGD([attaque],lr=0.005)

0 0.04686766862869263
1 3.540823459625244
2 9.229753494262695
3 15.51185417175293
4 22.443988800048828
5 28.232580184936523
6 33.313053131103516
7 38.62725830078125
8 39.327423095703125
9 44.154701232910156


80% des images "x+attaque" sont désormais mal classées ! (et le label de toutes à changer)
Pourtant, l'attaque ne se voit pas :

In [ ]:
with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x+attaque))
    _,z = z.max(1)
    print(z)

visu = torch.cat([x,x+attaque],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

Comment est ce que c'est possible ? Les réseaux ne sont pas du tout lipschitziens...

In [ ]:
with torch.no_grad():
    resnet = torchvision.models.resnet101(
        weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
    ).eval()
    resnet.fc = torch.nn.Identity()
    z = resnet(x)
    print(((z[0]-z[5])**2).sum())
    z_ = resnet(x+attaque)
    print(((z[0]-z_[0])**2).sum())

On voit que la représentation de l'image 0 devient presque aussi lointaine à cause de l'attaque que la distance avec l'image 5 !
Alors que nous ne voyons même pas la différence !

____________________________________________________________________________
=> retour aux slides (on revient ici après).
____________________________________________________________________________

# Attaque standard par patch "white box"
Maintenant on va regarder la création d'un patch adversarial : pour rappel, le problème des bruits invisibles c'est l'impossibilité de les faire dans le monde physique et l'existance de défense -> deux choses que les patches peuvent bypasser.

On va mettre un patch 36x36 en haut à gauche (remarquons que si le patch est juste noir, ça change rien).

In [ ]:
mask = torch.zeros(1,3,224,224)
mask[:,:,0:36,0:36] = 1

resnet = torchvision.models.resnet101(
    weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
).eval()
with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x*(1-mask)))
    _,z = z.max(1)
    print(z)

visu = torch.cat([x,x*(1-mask)],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

mais s'il est optimisé ?

In [ ]:
y = torch.Tensor([403, 405, 404, 405, 404,   4,   3,   3,   3,   2]).long()
cefunction = torch.nn.CrossEntropyLoss()
attaque = torch.nn.Parameter(torch.rand(1,3,224,224))
optimizer = torch.optim.SGD([attaque],lr=0.1)
for i in range(40):
  z = resnet(normalize(x*(1-mask)+mask*attaque))
  ce = cefunction(z,y)
  print(i,float(ce))
  ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
  optimizer.zero_grad()
  ce.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
      # l'attaque doit être dans le domaine image
      attaque = torch.clamp(attaque, 0,1)

  attaque = torch.nn.Parameter(attaque.clone())
  if i<20:
      optimizer = torch.optim.SGD([attaque],lr=0.1)
  else:
      optimizer = torch.optim.SGD([attaque],lr=0.05)

with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x*(1-mask) + attaque*mask))
    _,z = z.max(1)
    print(z)

visu = torch.cat([x,x*(1-mask)+attaque*mask],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

Des images ont vu leur label changé (c'est ici pas 100% mais c'est le MÊME patch pour toutes les images - et il n'est pas vraiment optimisé suffisamment longtemps).

## Limites

Ici on montre la facilité de faire une attaque **numérique** contre un réseau **naif** et **connu**.
Heureusement, la situation est très différente contre un réseau *défendu* ou *inconnu* ou quand l'attaque doit être *physiquement réalisable*.

### Réseaux défendus

On trouve très peu de réseaux défendus sur internet pour Imagenet (on trouve surtout des réseaux CIFAR et les rares qu'on peut trouver pour Imagenet comme dans le github https://github.com/MadryLab/robustness sont des réseaux custom).
Aussi, nous allons abandonnés nos avions/requins et faire des petites expériences sur CIFAR10.

Commençons par apprendre un réseau sur CIFAR:

In [ ]:
import torch
import torchvision

normalize = torchvision.transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),normalize])
trainset = torchvision.datasets.CIFAR10(
    root="build",
    train=True,
    download=True,
    transform=transform,
)

In [ ]:
resnet = torchvision.models.resnet18(
    weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1
).eval()
resnet.fc = torch.nn.Linear(512,10)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2
)
cefunction = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.0001)
meanloss = torch.zeros(50)
for i,(x,y) in enumerate(trainloader):
    z=resnet(x)
    loss = cefunction(z,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        meanloss[i%50]=loss.clone()
        if i%50==49:
          print(float(meanloss.mean()))
    if i==1000:
        break

vérifions que la perfo est pas trop mauvaise (très en dessous de l'état de l'art néanmoins car on a appris vraiment très très très peu) :

In [ ]:
testset = torchvision.datasets.CIFAR10(
    root="build",
    train=False,
    download=True,
    transform=transform,
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=500, shuffle=True, num_workers=2
)

with torch.no_grad():
    for x,y in testloader:
        z = resnet(x)
        _,z = z.max(1)
        good = (z==y).float().sum()
        print(float(good/5))
        break

c'est pas "terrible" mais ça ira pour la preuve de concept...

attaquons 10 images bien classée !

In [ ]:
I = [i for i in range(500) if y[i]==z[i]]
I = I[0:10]
y,x=y[I],x[I]
#ces 10 là sont bien classées !

cefunction = torch.nn.CrossEntropyLoss()
attaque = torch.nn.Parameter(torch.zeros(x.shape))
optimizer = torch.optim.SGD([attaque],lr=0.005)
for i in range(25):
  z = resnet(x+attaque)
  ce = cefunction(z,y)
  print(i,float(ce))
  ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
  optimizer.zero_grad()
  ce.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
    # l'attaque doit être invisible
    attaque = torch.clamp(attaque, -10./255,+10./255)

    # attaque+x doit être entre 0 et 1
    lowbound = -x
    uppbound = 1-x
    attaque = lowbound*(attaque<lowbound).float() + uppbound*(attaque>uppbound).float() + attaque *(attaque>=lowbound).float()*(attaque<=uppbound).float()

  attaque = torch.nn.Parameter(attaque.clone())
  optimizer = torch.optim.SGD([attaque],lr=0.005)

with torch.no_grad():
  z = resnet(x)
  _,z = z.max(1)
  print(z)
  z = resnet(x+attaque)
  _,z = z.max(1)
  print(z)

visu = torch.cat([x,x+attaque],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

Bon même si on est pas à 10/10, on voit que le modèle (pourtant non convergé) est quand même très sensible...

maintenant regardons si on apprend un modèle **robuste**.

In [ ]:
torch.save(resnet,"resnet.pth")
resnetrobuste = torch.load("resnet.pth") #force unrelated copy

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2
)
cefunction = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnetrobuste.parameters(), lr=0.0001)
meanloss = torch.zeros(50)
for i,(x,y) in enumerate(trainloader):
    #attack x, then update the weight to deal with the fact that z has been attacked
    attaque = torch.nn.Parameter(torch.zeros(x.shape))
    attackoptimizer = torch.optim.SGD([attaque],lr=0.001)
    for _ in range(10):
      z = resnet(x+attaque)
      ce = cefunction(z,y)
      ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
      optimizer.zero_grad()
      ce.backward()
      attaque.grad = attaque.grad.sign()
      optimizer.step()

    #now attaque is frozen
    with torch.no_grad():
        attaque = torch.Tensor(attaque.clone())

    z = resnetrobuste(x+attaque)
    loss = cefunction(z,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        meanloss[i%50]=loss.clone()
        if i%50==49:
          print(float(meanloss.mean()))

        if i%5==4:
          torch.save(resnetrobuste,"tmp.pth")
          resnet = torch.load("tmp.pth") #update the network from which attack is crafted
    if i==400:
        break

la performance sur les images "normales" devraient avoir baissée

In [ ]:
with torch.no_grad():
    for x,y in testloader:
        z = resnetrobuste(x)
        _,z = z.max(1)
        good = (z==y).float().sum()
        print(float(good/5))
        break

mais la performance devrait rester similaire sur des images attaquées

In [ ]:
I = [i for i in range(500) if y[i]==z[i]]
I = I[0:10]
y,x=y[I],x[I]
#ces 10 là sont bien classées !

cefunction = torch.nn.CrossEntropyLoss()
attaque = torch.nn.Parameter(torch.zeros(x.shape))
optimizer = torch.optim.SGD([attaque],lr=0.005)
for i in range(25):
  z = resnetrobuste(x+attaque)
  ce = cefunction(z,y)
  print(i,float(ce))
  ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
  optimizer.zero_grad()
  ce.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
    # l'attaque doit être invisible
    attaque = torch.clamp(attaque, -10./255,+10./255)

    # attaque+x doit être entre 0 et 1
    lowbound = -x
    uppbound = 1-x
    attaque = lowbound*(attaque<lowbound).float() + uppbound*(attaque>uppbound).float() + attaque *(attaque>=lowbound).float()*(attaque<=uppbound).float()

  attaque = torch.nn.Parameter(attaque.clone())
  optimizer = torch.optim.SGD([attaque],lr=0.005)

with torch.no_grad():
  z = resnetrobuste(x)
  _,z = z.max(1)
  print(z)
  z = resnetrobuste(x+attaque)
  _,z = z.max(1)
  print(z)

visu = torch.cat([x,x+attaque],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

On voit que le modèle est déjà plus robuste (alors qu'on a effectué que très très peu d'itération de robustification)...

**De fait aujourd'hui, ce type d'apprentissage permet d'avoir une robustesse forte contre les attaques invisibles.**

### Transferabilité

Une des autres difficultés est d'attaquer un réseau inconnu : revenons à nos images d'avions et requins et recréons notre attaque invisible du début



In [ ]:
x = [torchvision.io.read_image(str(i)+".png") for i in range(10)]
x = torch.stack(x,dim=0).float()/255
y = torch.Tensor([403, 405, 404, 405, 404,   4,   3,   3,   3,   2]).long()

normalize = torchvision.transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
resnet = torchvision.models.resnet101(
    weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
).eval()

cefunction = torch.nn.CrossEntropyLoss()
attaque = torch.nn.Parameter(torch.zeros(x.shape))
optimizer = torch.optim.SGD([attaque],lr=0.005)
for i in range(10):
  z = resnet(normalize(x+attaque))
  ce = cefunction(z,y)
  print(i,float(ce))
  ce = -ce # on veut MAXIMISER la cross entropy puisqu'on attaque
  optimizer.zero_grad()
  ce.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
      # l'attaque doit être invisible
      attaque = torch.clamp(attaque, -10./255,+10./255)

      # attaque+x doit être entre 0 et 1
      lowbound = -x
      uppbound = 1-x
      attaque = lowbound*(attaque<lowbound).float() + uppbound*(attaque>uppbound).float() + attaque *(attaque>=lowbound).float()*(attaque<=uppbound).float()

  attaque = torch.nn.Parameter(attaque.clone())
  optimizer = torch.optim.SGD([attaque],lr=0.005)


with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x+attaque))
    _,z = z.max(1)
    print(z)

et regardons ce que donne cette attaque sur un autre réseau

In [ ]:
efficientnet = torchvision.models.efficientnet_b0(
    weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
).eval()

with torch.no_grad():
    z = efficientnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = efficientnet(normalize(x+attaque))
    _,z = z.max(1)
    print(z)

on voit que l'attaque n'a presque aucun effet !!!

essayons de rendre l'attaque transferable en attaquant non pas la classification mais les features comme proposé par https://openaccess.thecvf.com/content_CVPR_2019/papers/Inkawhich_Feature_Space_Perturbations_Yield_More_Transferable_Adversarial_Examples_CVPR_2019_paper.pdf

In [ ]:
resnet = torchvision.models.resnet101(
    weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
).eval()
resnet.fc = torch.nn.Identity()
with torch.no_grad():
  f_0 = resnet(normalize(x))
  f_0 = torch.stack([f_0[-1]]*5+[f_0[0]]*5,dim=0)

attaque = torch.nn.Parameter(torch.zeros(x.shape))
optimizer = torch.optim.SGD([attaque],lr=0.005)
for i in range(10):
  f = resnet(normalize(x+attaque))

  loss = ((f-f_0)**2).sum()
  print(i,float(loss))
  optimizer.zero_grad()
  loss.backward()
  attaque.grad = attaque.grad.sign()
  optimizer.step()
  with torch.no_grad():
      # l'attaque doit être invisible
      attaque = torch.clamp(attaque, -10./255,+10./255)

      # attaque+x doit être entre 0 et 1
      lowbound = -x
      uppbound = 1-x
      attaque = lowbound*(attaque<lowbound).float() + uppbound*(attaque>uppbound).float() + attaque *(attaque>=lowbound).float()*(attaque<=uppbound).float()

  attaque = torch.nn.Parameter(attaque.clone())
  optimizer = torch.optim.SGD([attaque],lr=0.005)


resnet = torchvision.models.resnet101(
    weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
).eval()
with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x+attaque))
    _,z = z.max(1)
    print(z)

resnet = torchvision.models.resnet50(
    weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1
).eval()
with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x+attaque))
    _,z = z.max(1)
    print(z)

On voit apparaitre une très légère transférabilité...

Pour les attaques invisibles, il faudrait rajouter un générateur mais cela est difficilement démontrable sur google collab

Pour aller plus loin sur la transferabilité des patches on pourrait tenter une attaque "loss transport" comme dans https://openreview.net/forum?id=nZP10evtkV
On peut néanmoins évaluer les patches produits.

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt

x = [torchvision.io.read_image(str(i)+".png") for i in range(10)]
x = torch.stack(x,dim=0).float()/255

SHARK, PLANE = [2, 3, 4], [403, 404, 405]
normalize = torchvision.transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
resnet = torchvision.models.resnet101(
    weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
).eval()

S = 80
mask = torch.zeros(1,3,224,224)
mask[:,:,0:S,0:S] = 1

attaque = torchvision.io.read_image("patch_bannane.png").float()/255
attaque = torch.nn.functional.interpolate(attaque.view(1,3,297,295), size=(S,S))
tmp = torch.zeros(1,3,224,224)
tmp[:,:,0:S,0:S] = attaque
attaque = tmp

resnet = torchvision.models.resnet101(
    weights=torchvision.models.ResNet101_Weights.IMAGENET1K_V1
).eval()
with torch.no_grad():
    z = resnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x*(1-mask)))
    _,z = z.max(1)
    print(z)
    z = resnet(normalize(x*(1-mask) + mask*attaque))
    _,z = z.max(1)
    print(z)

efficientnet = torchvision.models.efficientnet_b0(
    weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
).eval()
with torch.no_grad():
    z = efficientnet(normalize(x))
    _,z = z.max(1)
    print(z)
    z = efficientnet(normalize(x*(1-mask)))
    _,z = z.max(1)
    print(z)
    z = efficientnet(normalize(x*(1-mask) + mask*attaque))
    _,z = z.max(1)
    print(z)

visu = torch.cat([x,x*(1-mask),x*(1-mask)+mask*attaque ],dim=0)
visu = torchvision.utils.make_grid(visu, nrow=5)
plt.imshow(visu.permute(1, 2, 0).numpy())
plt.show()

on a bien 1 patch (assez gros mais "imprimable") qui arrive à casser quelques images sur 2 réseaux différents !

**==> c'est la fin de ce TP**